In [11]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Data
from azure.identity import DefaultAzureCredential
from azure.ai.ml.constants import AssetTypes

import pandas as pd
import lightgbm
from sklearn.model_selection import train_test_split
import mlflow

In [12]:
# If error is thrown with below cell because config.json is not found, os.chdir may solve the problem.
import os
os.chdir(path="/mnt/batch/tasks/shared/LS_root/mounts/clusters/shuitcpueastus01/code/Users/shuit/ak_handson")

In [13]:
ml_client = MLClient.from_config(credential=DefaultAzureCredential(exclude_shared_token_cache_credential=True),
                     logging_enable=True)

Found the config file in: ./config.json


In [14]:
azureml_mlflow_uri = ml_client.workspaces.get(ml_client.workspace_name).mlflow_tracking_uri
mlflow.set_tracking_uri(azureml_mlflow_uri)

In [15]:
experiment_name = 'nyc_taxi_regression_notebook'
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='', creation_time=1664779291782, experiment_id='fefbcf4c-261e-44d9-aaaf-70959ff96351', last_update_time=None, lifecycle_stage='active', name='nyc_taxi_regression_notebook', tags={}>

In [16]:
#origin_df = pd.read_csv("data/nyc_taxi_dataset.csv")

In [17]:
#train, test = train_test_split(origin_df, test_size=0.2, random_state=1234)
train = pd.read_csv("data/nyc_taxi_train_dataset.csv")
test = pd.read_csv("data/nyc_taxi_test_dataset.csv")

x_train = train[train.columns[train.columns != "totalAmount"]]
y_train = train["totalAmount"]

x_test = test[test.columns[test.columns != "totalAmount"]]
y_test = test["totalAmount"]

In [18]:
params = {
        "boosting_type": "gbdt",
        "metric": "rmse",
        "learning_rate": 0.1,
        "num_leaves": 10,
        "min_data_in_leaf": 2,
        "num_iteration": 100,
        "task": "train",
        "objective": "regression",
    }

In [19]:
with mlflow.start_run() as run:
    mlflow.lightgbm.autolog(registered_model_name="nyc_taxi_regressor_lightgbm")
    train_dataset = lightgbm.Dataset(x_train, y_train)
    test_dataset = lightgbm.Dataset(x_test, y_test, reference=train_dataset)
    gbm = lightgbm.train(params, train_dataset, num_boost_round=50, valid_sets=test_dataset, early_stopping_rounds=10)

/anaconda/envs/azureml_ak_handson_env/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/anaconda/envs/azureml_ak_handson_env/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000186 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1540
[LightGBM] [Info] Number of data points in the train set: 4818, number of used features: 8
[LightGBM] [Info] Start training from score 14.254768


2022/10/03 06:46:21 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/anaconda/envs/azureml_ak_handson_env/lib/python3.10/site-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
Registered model 'nyc_taxi_regressor_lightgbm' already exists. Creating a new version of this model...
2022/10/03 06:46:24 INFO m